In [2]:
import pandas as pd
import yfinance as yf
from collections.abc import Collection
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np
from datetime import datetime, date, timedelta
import ast
import matplotlib.pyplot as plt
import statsmodels.api as sm
import requests

In [3]:
# TODO
# 1) this probably should pull stored values esp for back testing
# 2) maybe this should record the earliest date both assets have a price
def get_adj_close_history_df(tickers: list[str], start_date: datetime, end_date: datetime) -> pd.DataFrame:
    price_column_name_suffix = '_price'
    prices_df = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=True
    )['Close']
    prices_df.columns = [f"{col_name + price_column_name_suffix}" for col_name in prices_df.columns]
    return prices_df.dropna()

def create_returns_from_price_history(price_history_df: pd.DataFrame) -> float:
    pair_price_returns_df = price_history_df.pct_change() * 100
    pair_price_returns_df.columns = pair_price_returns_df.columns.str.replace(r'_price$', '_1d_returns', regex=True)
    return pair_price_returns_df

def calculate_regression_coefficient(Y: pd.Series, X: pd.Series, x_label: str) -> float:
    X = sm.add_constant(X)
    results = sm.OLS(Y, X).fit()
    return float(results.params[x_label])

def calculate_historical_beta_at_date(ticker: str, cur_date: datetime) -> float:
    benchmark_ticker = 'IWV' # russell 3000 etf
    beta_calculation_window_in_days = 365 * 3 + 1 # adding one to get returns for full period
    beta_calculation_period_start = cur_date - timedelta(days=beta_calculation_window_in_days)
    prices_df = get_adj_close_history_df(
        [ticker, benchmark_ticker],
        beta_calculation_period_start,
        cur_date,
    )
    returns_df = create_returns_from_price_history(prices_df).dropna()
    returns_df.columns = returns_df.columns.str.replace(r'_1d_returns', '', regex=True)
    return calculate_regression_coefficient(returns_df[ticker], returns_df[benchmark_ticker], benchmark_ticker)

def is_price_series_integrated_of_order_one(
        price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    p_level = adfuller(price_series)[1]
    # Test if series is stationary
    if p_level < pvalue_threshold:
        return False

    # Test if series is stationary after applying one difference
    p_diff = adfuller(price_series.diff().dropna())[1]
    return True if p_diff < pvalue_threshold else False


def is_pair_engle_granger_cointegrated(
        ticker1_price_series: pd.Series,
        ticker2_price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    _, pvalue, _ = coint(ticker1_price_series, ticker2_price_series)
    return True if pvalue < pvalue_threshold else False

# TODO underwrite det_order and k_ar_diff parameter values
# current parameter values are from chatgpt
# also corroborated here https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/
def is_pair_johansen_cointegrated(
        pair_price_history_df: pd.DataFrame
) -> bool:
    johansen_cointegration_result = coint_johansen(
        pair_price_history_df.values,
        det_order=0,
        k_ar_diff=1
    )
    index_of_95_pct_confidence_level_critical_value = 1
    index_of_trace_stat = 0
    trace_stat = johansen_cointegration_result.lr1[index_of_trace_stat]
    critical_value = johansen_cointegration_result.cvt[
        0,
        index_of_95_pct_confidence_level_critical_value
    ]
    return trace_stat > critical_value


def are_historical_betas_close_enough(as_of_date: datetime, ticker_one: str, ticker_two: str) -> bool:
    beta_absolute_difference_threshold = 0.3
    ticker_one_beta = calculate_historical_beta_at_date(ticker_one, as_of_date)
    ticker_two_beta = calculate_historical_beta_at_date(ticker_two, as_of_date)
    is_close_enough = np.isclose(ticker_one_beta, ticker_two_beta, atol=beta_absolute_difference_threshold, rtol=0)
    return bool(is_close_enough)  

# NOTE: This methodology (ex beta filter) came from Caldeira & Caldeira 2013. Paper is in references folder
def is_pair_tradable(
        ticker_pair: tuple[str],
        start_date: datetime,
        end_date: datetime
) -> bool:
    try:
        if not are_historical_betas_close_enough(end_date, ticker_pair[0], ticker_pair[1]):
            print(f"Pair {ticker_pair} betas are not close enough")
            return False
        pair_price_history_df = get_adj_close_history_df(ticker_pair, start_date, end_date)
        ticker1_price_series = pair_price_history_df.iloc[:,0]
        ticker2_price_series = pair_price_history_df.iloc[:,1]
        if not is_price_series_integrated_of_order_one(ticker1_price_series):
            print(f"Pair {ticker_pair} first ticker series is not I(1)")
            return False
        if not is_price_series_integrated_of_order_one(ticker2_price_series):
            print(f"Pair {ticker_pair} second ticker series is not I(1)")
            return False
        if not is_pair_engle_granger_cointegrated(ticker1_price_series, ticker2_price_series):
            print(f"Pair {ticker_pair} is not engle granger cointegrated")
            return False
        if not is_pair_johansen_cointegrated(pair_price_history_df):
            print(f"Pair {ticker_pair} is not johansen cointegrated")
            return False
        return True
    except Exception as e:
        print(f"Unable to evaluate pair {ticker_pair}")
        print(e)
        return False

def calculate_historical_gamma(
        pair_price_history_df: pd.DataFrame, 
        in_sample_start_date: datetime, 
        in_sample_end_date: datetime
) -> float:
    """
    Estimate gamma via OLS and return the spread = P1 - gamma * P2
    """
    date_mask = (pair_price_history_df.index >= in_sample_start_date) & (pair_price_history_df.index <= in_sample_end_date)
    pair_price_history_in_sample_df = pair_price_history_df[date_mask] 
    ticker1_price_series = pair_price_history_in_sample_df.iloc[:,0]
    ticker2_price_series = pair_price_history_in_sample_df.iloc[:,1]

    # Add constant to allow intercept in regression
    X = sm.add_constant(ticker2_price_series)
    model = sm.OLS(ticker1_price_series, X).fit()

    return float(model.params[1])

def calculate_spread(
        pair_price_history_df: pd.DataFrame,
        gamma: float,
        start_date: datetime, 
        end_date: datetime
) -> pd.Series:
    date_mask = (pair_price_history_df.index >= start_date) & (pair_price_history_df.index <= end_date)
    pair_price_history_out_of_sample_df = pair_price_history_df[date_mask] 
    ticker1_price_series = pair_price_history_out_of_sample_df.iloc[:,0]
    ticker2_price_series = pair_price_history_out_of_sample_df.iloc[:,1]

    return ticker1_price_series - gamma * ticker2_price_series

def calculate_trailing_zscore(spread: pd.Series, z_score_window_in_days: int, start_date: datetime, end_date: datetime) -> pd.Series:
    rolling_mean = spread.rolling(window=z_score_window_in_days).mean()
    rolling_std = spread.rolling(window=z_score_window_in_days).std()
    zscore_series = (spread - rolling_mean) / rolling_std
    zscore_filtered_series = zscore_series.loc[start_date:end_date]
    return zscore_filtered_series


# TODO add time series of prices on this graph for additional clarity
def plot_zscore_zeries(zscore_series: pd.Series):
    plt.figure(figsize=(14,6))
    plt.plot(zscore_series.index, zscore_series.values, label='Z-Score', color='blue')

    # Horizontal lines for 1 and 2 standard deviations
    for level in [1, 2]:
        plt.axhline(level, color='gray', linestyle='--', linewidth=1, label=f'+{level}σ' if level==1 else None)
        plt.axhline(-level, color='gray', linestyle='--', linewidth=1, label=f'-{level}σ' if level==1 else None)

    # Only show one label per line for legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.title('Trailing Z-Score of Spread (Out-of-Sample)')
    plt.xlabel('Date')
    plt.ylabel('Z-Score')
    plt.grid(True)
    plt.show()

def get_ticker_pairs() -> list[tuple]:
    ticker_df = pd.read_csv('../data/russell_3000_constituents.csv')
    tickers = ticker_df['Ticker'].to_list()
    pairs = []
    for ticker_one in tickers:
        for ticker_two in tickers:
            if ticker_one == ticker_two:
                continue
            pairs.append((ticker_one, ticker_two))
    return pairs
    
# TODO should eventually create a generic abstraction that takes
# a universe (list of tickers) and returns a collection of tradable pairs.
# But that computation is time consuming and I already have a list of candidate pairs
# so I'm going to start there for now.

In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import os

valid_pairs = []
seen_pairs = []
start_date = datetime(2022,1,1)
end_date = datetime(2023,12,31)


pairs = get_ticker_pairs()

for pair in pairs:
    if sorted(pair) in seen_pairs:
        continue
    if is_pair_tradable(pair, start_date, end_date):
        print("VALID:", pair)
        valid_pairs.append(pair)
        pd.DataFrame({"Valid Pairs": [pair]}).to_csv(
                    '../data/valid_sp500_pairs.csv',
                    mode="a",      # append mode
                    header=False,
                    index=False
                )
    seen_pairs.append(sorted(pair))


Pair ('NVDA', 'MSFT') betas are not close enough
Pair ('NVDA', 'AAPL') betas are not close enough
Pair ('NVDA', 'AMZN') betas are not close enough
Pair ('NVDA', 'META') betas are not close enough
Pair ('NVDA', 'AVGO') betas are not close enough
Pair ('NVDA', 'GOOGL') betas are not close enough
Pair ('NVDA', 'GOOG') betas are not close enough
Pair ('NVDA', 'TSLA') is not engle granger cointegrated



1 Failed download:
['BRKB']: YFTzMissingError('possibly delisted; no timezone found')


Unable to evaluate pair ('NVDA', 'BRKB')
zero-size array to reduction operation maximum which has no identity
Pair ('NVDA', 'JPM') betas are not close enough
Pair ('NVDA', 'LLY') betas are not close enough
Pair ('NVDA', 'V') betas are not close enough
Pair ('NVDA', 'NFLX') betas are not close enough
Pair ('NVDA', 'XOM') betas are not close enough
Pair ('NVDA', 'ORCL') betas are not close enough
Pair ('NVDA', 'MA') betas are not close enough
Pair ('NVDA', 'WMT') betas are not close enough
Pair ('NVDA', 'COST') betas are not close enough
Pair ('NVDA', 'JNJ') betas are not close enough
Pair ('NVDA', 'HD') betas are not close enough
Pair ('NVDA', 'ABBV') betas are not close enough
Pair ('NVDA', 'PLTR') is not engle granger cointegrated
Pair ('NVDA', 'PG') betas are not close enough
Pair ('NVDA', 'BAC') betas are not close enough
Pair ('NVDA', 'GE') betas are not close enough
Pair ('NVDA', 'CVX') betas are not close enough
Pair ('NVDA', 'UNH') betas are not close enough
Pair ('NVDA', 'CSCO'

HTTP Error 404: 

1 Failed download:
['XTSLA']: YFTzMissingError('possibly delisted; no timezone found')


Unable to evaluate pair ('NVDA', 'XTSLA')
zero-size array to reduction operation maximum which has no identity
Pair ('NVDA', 'CAT') betas are not close enough
Pair ('NVDA', 'MCD') betas are not close enough
Pair ('NVDA', 'RTX') betas are not close enough
Pair ('NVDA', 'DIS') betas are not close enough
Pair ('NVDA', 'MRK') betas are not close enough
Pair ('NVDA', 'T') betas are not close enough
Pair ('NVDA', 'NOW') betas are not close enough
Pair ('NVDA', 'PEP') betas are not close enough
Pair ('NVDA', 'UBER') betas are not close enough
Pair ('NVDA', 'C') betas are not close enough
Pair ('NVDA', 'MU') betas are not close enough
Pair ('NVDA', 'AXP') betas are not close enough
Pair ('NVDA', 'INTU') betas are not close enough
Pair ('NVDA', 'QCOM') betas are not close enough
Pair ('NVDA', 'VZ') betas are not close enough
Pair ('NVDA', 'MS') betas are not close enough
Pair ('NVDA', 'TMO') betas are not close enough
Pair ('NVDA', 'BKNG') betas are not close enough
Pair ('NVDA', 'BLK') betas a

...

Unable to evaluate pair ('GOOGL', 'LOAR')
zero-size array to reduction operation maximum which has no identity
Pair ('GOOGL', 'AAP') betas are not close enough
Pair ('GOOGL', 'SHAK') is not engle granger cointegrated
Pair ('GOOGL', 'WNS') betas are not close enough
Pair ('GOOGL', 'AKRO') betas are not close enough
Pair ('GOOGL', 'TBBK') is not engle granger cointegrated
Pair ('GOOGL', 'CALX') betas are not close enough



1 Failed download:
['AMTM']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-12-30 00:00:00 -> 2023-12-31 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1609304400, endDate = 1703998800")')


Unable to evaluate pair ('GOOGL', 'AMTM')
zero-size array to reduction operation maximum which has no identity
Pair ('GOOGL', 'FUL') is not engle granger cointegrated
Pair ('GOOGL', 'HCC') betas are not close enough
Pair ('GOOGL', 'BXMT') is not engle granger cointegrated
Pair ('GOOGL', 'NUVL') betas are not close enough
Pair ('GOOGL', 'MSGS') betas are not close enough
Pair ('GOOGL', 'CACC') first ticker series is not I(1)
Pair ('GOOGL', 'CDP') betas are not close enough
Pair ('GOOGL', 'FHB') betas are not close enough
Pair ('GOOGL', 'ENVA') is not engle granger cointegrated
Pair ('GOOGL', 'MZTI') betas are not close enough
Pair ('GOOGL', 'NVST') betas are not close enough
Pair ('GOOGL', 'HASI') is not engle granger cointegrated
Pair ('GOOGL', 'TPH') is not engle granger cointegrated
Pair ('GOOGL', 'NHI') betas are not close enough
Pair ('GOOGL', 'LGND') second ticker series is not I(1)
Pair ('GOOGL', 'BCC') is not engle granger cointegrated
Pair ('GOOGL', 'REVG') is not engle granger


1 Failed download:
['KRMN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-12-30 00:00:00 -> 2023-12-31 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1609304400, endDate = 1703998800")')


Unable to evaluate pair ('GOOGL', 'KRMN')
zero-size array to reduction operation maximum which has no identity
Pair ('GOOGL', 'PII') is not engle granger cointegrated
Pair ('GOOGL', 'PENN') betas are not close enough
Pair ('GOOGL', 'KAR') second ticker series is not I(1)
Pair ('GOOGL', 'DRS') betas are not close enough
Pair ('GOOGL', 'WD') is not engle granger cointegrated
Pair ('GOOGL', 'VAL') is not engle granger cointegrated
Pair ('GOOGL', 'CHH') betas are not close enough
Pair ('GOOGL', 'CRNX') is not engle granger cointegrated
Pair ('GOOGL', 'BL') betas are not close enough
Pair ('GOOGL', 'NMRK') is not engle granger cointegrated
Pair ('GOOGL', 'CNK') is not engle granger cointegrated
Pair ('GOOGL', 'YETI') betas are not close enough
Pair ('GOOGL', 'YOU') betas are not close enough
Pair ('GOOGL', 'HAYW') second ticker series is not I(1)
Pair ('GOOGL', 'MGNI') betas are not close enough
Pair ('GOOGL', 'BANF') betas are not close enough
Pair ('GOOGL', 'APLE') first ticker series is 

In [ ]:
in_sample_start_date = datetime(2022,1,1)
in_sample_end_date = datetime(2023,12,31)

z_score_window_in_days = 50

out_of_sample_start_date = in_sample_end_date + timedelta(days=1)
out_of_sample_end_date = datetime(2025,1,1)

df = pd.read_csv('../data/valid_sp500_pairs.csv', header=None, names=['Pairs'])
df['Pairs'] = df['Pairs'].apply(ast.literal_eval)


for ticker_pair in df['Pairs'].to_list()[:1]:

    # Get prices once for entire evaluation period
    pair_price_history_df = get_adj_close_history_df(ticker_pair, in_sample_start_date, out_of_sample_end_date)
    # calculate gamma over period where cointegration has been validated
    gamma = calculate_historical_gamma(pair_price_history_df, in_sample_start_date, in_sample_end_date)

    # Lagging the out of sample start date is to ensure that there are z scores for the first day of the out of sample period
    #TODO there is a bug here bc pair price history is in trading days but the timedelta calculates in calendar days.
    # doubling the window fixed it for now but it is not precise. 
    spread_series = calculate_spread(
        pair_price_history_df,
        gamma,
        out_of_sample_start_date - timedelta(days=z_score_window_in_days*2),
        out_of_sample_end_date
    )
    out_of_sample_z_score_series = calculate_trailing_zscore(
        spread_series,
        z_score_window_in_days,
        out_of_sample_start_date,
        out_of_sample_end_date
    )
    plot_zscore_zeries(out_of_sample_z_score_series)

In [ ]:

def is_exit_condition_met(
        spread_z_score_series: pd.Series,
        t_minus_one_date: datetime,
        t_minus_two_date: datetime,
        exit_threshold: float
) -> bool:
    z_score_at_t_minus_one_date = spread_z_score_series.loc[t_minus_one_date]
    z_score_at_t_minus_two_date = spread_z_score_series.loc[t_minus_two_date]
    # upwards mean reversion across exit threshold
    if z_score_at_t_minus_one_date > exit_threshold and z_score_at_t_minus_two_date < exit_threshold:
        return True
    # downwards mean reversion across exit threshold
    if z_score_at_t_minus_one_date < exit_threshold and z_score_at_t_minus_two_date > exit_threshold:
        return True
    if np.isclose(z_score_at_t_minus_one_date, exit_threshold, atol=0.1):
        return True
    return False


def get_date_of_previous_day(
        spread_z_score_series: pd.Series,
        index: int,
) -> datetime:
    return spread_z_score_series.index[index - 1]


def set_weights_to_zero_at_current_date(
        weight_at_date_df: pd.DataFrame,
        date: datetime,
        ticker_one_weight_column_name: str,
        ticker_two_weight_column_name: str,
) -> pd.DataFrame:
    weight_at_date_df.loc[date, ticker_one_weight_column_name] = 0
    weight_at_date_df.loc[date, ticker_two_weight_column_name] = 0
    return weight_at_date_df


def enter_position_at_current_date(
        weight_at_date_df: pd.DataFrame,
        date: datetime,
        ticker_one_weight_column_name: str,
        ticker_two_weight_column_name: str,
        *,
        is_long_ticker_one: bool
)  -> pd.DataFrame:
    long_position_flag = 1
    short_position_flag = -1
    if is_long_ticker_one:
        weight_at_date_df.loc[date, ticker_one_weight_column_name] = long_position_flag
        weight_at_date_df.loc[date, ticker_two_weight_column_name] = short_position_flag
    else:
        weight_at_date_df.loc[date, ticker_one_weight_column_name] = short_position_flag
        weight_at_date_df.loc[date, ticker_two_weight_column_name] = long_position_flag
    return weight_at_date_df


def assign_weights_from_prev_date_to_cur_date(
        weight_at_date_df: pd.DataFrame,
        cur_date: datetime,
        t_minus_one_date: datetime,
) -> pd.DataFrame:
    weight_at_date_df.loc[cur_date] = weight_at_date_df.loc[t_minus_one_date]
    return weight_at_date_df


def create_trade_signals_for_spread_z_score_series(
        ticker_pair: tuple[str,str],
        spread_z_score_series: pd.Series,
) -> pd.DataFrame:
    ticker_one = ticker_pair[0]
    ticker_two = ticker_pair[1]
    weight_column_name_suffix = '_trade_side'
    ticker_one_weight_column_name = ticker_one + weight_column_name_suffix
    ticker_two_weight_column_name = ticker_two + weight_column_name_suffix

    weight_at_date_df = pd.DataFrame(
        index=spread_z_score_series.index,
        columns=[ticker_one_weight_column_name, ticker_two_weight_column_name]
    )
    entrance_threshold = 2
    exit_threshold = 0

    is_invested = False
    for i, (cur_date, z_score) in enumerate(spread_z_score_series.items()):
        # TODO there is a better way than to ignore the first two days. 
        # probably just extend the spread_z_score_series
        if i < 2:
            continue
        t_minus_one_date = spread_z_score_series.index[i-1]
        t_minus_two_date = spread_z_score_series.index[i-2]
        z_score_at_t_minus_one_date = spread_z_score_series.loc[t_minus_one_date]
    
        if z_score_at_t_minus_one_date >= entrance_threshold and not is_invested:
            weight_at_date_df = enter_position_at_current_date(
                weight_at_date_df,
                cur_date,
                ticker_one_weight_column_name,
                ticker_two_weight_column_name,
                is_long_ticker_one=True
            )
            is_invested = True

        elif is_invested and is_exit_condition_met(spread_z_score_series, t_minus_one_date, t_minus_two_date, exit_threshold):
            weight_at_date_df = set_weights_to_zero_at_current_date(
                                    weight_at_date_df,
                                    cur_date,
                                    ticker_one_weight_column_name,
                                    ticker_two_weight_column_name
                                )
            is_invested = False
        
        elif z_score_at_t_minus_one_date <= -entrance_threshold and not is_invested:
            weight_at_date_df = enter_position_at_current_date(
                weight_at_date_df,
                cur_date,
                ticker_one_weight_column_name,
                ticker_two_weight_column_name,
                is_long_ticker_one=False
            )
            is_invested = True
        else:
            weight_at_date_df = assign_weights_from_prev_date_to_cur_date(weight_at_date_df, cur_date, t_minus_one_date)
    
    return weight_at_date_df.fillna(0)

In [21]:
trade_signals_df = create_trade_signals_for_spread_z_score_series(ticker_pair, out_of_sample_z_score_series)
# pair_price_returns_df = create_returns_from_price_history(pair_price_history_df)

NameError: name 'create_trade_signals_for_spread_z_score_series' is not defined

In [22]:
appl_beta = calculate_historical_beta_at_date('AAPL', datetime(2025,1,1))
# msft_beta = calculate_historical_beta_at_date('MSFT', datetime(2025,1,1))

const    0.017696
IWV      1.158348
dtype: float64


In [51]:
df = pd.read_csv('../data/russell_3000_constituents.csv')

In [53]:
df['Ticker']

0         NVDA
1         MSFT
2         AAPL
3         AMZN
4         META
5         AVGO
6        GOOGL
7         GOOG
8         TSLA
9         BRKB
10         JPM
11         LLY
12           V
13        NFLX
14         XOM
15        ORCL
16          MA
17         WMT
18        COST
19         JNJ
20          HD
21        ABBV
22        PLTR
23          PG
24         BAC
25          GE
26         CVX
27         UNH
28        CSCO
29         WFC
30          KO
31         AMD
32          PM
33         IBM
34          GS
35         ABT
36         CRM
37         LIN
38       XTSLA
39         CAT
40         MCD
41         RTX
42         DIS
43         MRK
44           T
45         NOW
46         PEP
47        UBER
48           C
49          MU
50         AXP
51        INTU
52        QCOM
53          VZ
54          MS
55         TMO
56        BKNG
57         BLK
58         GEV
59         TXN
60        LRCX
61        SCHW
62          BA
63         TJX
64        ADBE
65        ISRG
66        

In [47]:
are_historical_betas_close_enough(datetime(2025,1,1), 'MSFT', 'AAPL')

True

In [49]:
ticker_one_beta = calculate_historical_beta_at_date('MSFT', datetime(2025,1,1))
ticker_two_beta = calculate_historical_beta_at_date('TSLA', datetime(2025,1,1))

ticker_two_beta

1.929817559441993

In [ ]:
# the initial idea was to equal beta weight but I stupidly did not realize that I could end up with materially
# lob-sided trades if the betas of the two assets were not roughly equal which means the trade would not
# accurately capture the reversion of the spread. Hence, it seems that a prerequisite for pairing be that
# each asset have similar betas

def calculate_historical_betas(trade_signals_df: pd.DataFrame) -> pd.DataFrame:
    benchmark_ticker = 'IWV' # russell 3000 etf
    beta_calculation_window_in_days = 365 * 3 + 1
    column_names_with_embedded_tickers = list(trade_signals_df.columns)
    tickers = [
        column_name.split('_')[0]
        for column_name in column_names_with_embedded_tickers
    ]
    tickers.append(benchmark_ticker)

    start_date = trade_signals_df.index[0] - timedelta(days=beta_calculation_window_in_days)
    end_date = trade_signals_df.index[-1]

    prices_df = get_adj_close_history_df(
        tickers + [benchmark_ticker],
        start_date,
        end_date,
    )

    returns_df = create_returns_from_price_history(prices_df).dropna()
    benchmark_returns = returns_df[benchmark_ticker+'_1d_returns']
    betas = {}
    for ticker in tickers[:-1]:
        y = returns_df[ticker+'_1d_returns']
        X = sm.add_constant(benchmark_returns)
        model = sm.OLS(y, X).fit()
        betas[ticker] = model.params[1]  # slope = beta

    return pd.DataFrame.from_dict(betas, orient="index", columns=["beta"])


In [211]:
calculate_historical_betas(trade_signals_df).head(20)

/var/folders/9t/v6gpfcbs71bcpv1bsh004rrc0000gn/T/ipykernel_63239/2590961576.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[ticker] = model.params[1]  # slope = beta
/var/folders/9t/v6gpfcbs71bcpv1bsh004rrc0000gn/T/ipykernel_63239/2590961576.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[ticker] = model.params[1]  # slope = beta


,beta
MMM,0.722382
LNT,0.394200
